In this Cell we drop 'url', 'urlid', 'IP' and 'region' and 'bidid' from the training data.

In [ ]:
import os
import csv
import numpy as np
import random
from random import shuffle
import math
import pandas as pd
from random import uniform
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score


filename = '../input/train.csv'

data = pd.read_csv(filename)


train = data.drop(['bidid',  'region',  'bidprice','url','urlid', 'IP'], axis=1)


train['slotvisibility'] = pd.to_numeric(train['slotvisibility'], errors='coerce')
avgSlot = train['slotvisibility'].mean()

vals = '../input/validation.csv'

valid = pd.read_csv(vals)


valid = valid.drop(['bidid',  'region',  'bidprice','url','urlid','IP'], axis=1)
valid['slotvisibility'] = pd.to_numeric(valid['slotvisibility'], errors='coerce')



fn = '../input/test.csv'

tests = pd.read_csv(fn)

tests = tests.drop(['region','url','urlid','IP'], axis=1)


small = train[train.click==1]
big = train[train.click==0]






Negatively down-sample training

In [ ]:
nsamp = int((len(big)*0.2))

nBig = resample(big, replace=False, n_samples=nsamp, random_state=123) 

separate out clicks and payprice from feature data

In [ ]:
train=nBig.append(small, ignore_index=True)


ids = tests['bidid']
tests = tests.drop(['bidid'], axis=1)
clicks = train['click']
train = train.drop(['click'], axis=1)
price = train['payprice']
train = train.drop(['payprice'], axis=1)



cVal = valid['click']
valid = valid.drop(['click'], axis=1)
vPrice = valid['payprice']
valid = valid.drop(['payprice'], axis=1)

append test and validation before feature engineering, ensure all go through same tranformation

In [ ]:
train=train.append(valid, ignore_index=True)
train=train.append(tests, ignore_index=True)

In [ ]:
from tqdm import tqdm
from sklearn import preprocessing
def getDict(train):
    
    tagD = {}

    for index, row in tqdm(train.iterrows()):
        for j in [x.strip() for x in row['usertag'].split(',')]:
            if j in tagD:
                tagD[j].append(index)
            else:
                tagD[j]=[index]
    
    
    return tagD




get indexes of where each usertag is present

In [ ]:
train['usertag']=train['usertag'].fillna("")
tagD = getDict(train)

tagD.pop('', None)
    
    
for i in tagD:
    train[i]=0




one-hot usertags based off collected index information

In [ ]:



for i in tqdm(tagD.keys()):

    train.iloc[tagD[i], train.columns.get_loc(i)] = 1


In [ ]:
train = train.drop(['usertag'], axis=1)

In [ ]:
from tqdm import tqdm
def transformDatas(df, avgSlot):
    
    df = df.replace("FourthView","4")
    df = df.replace("OtherView","0")
    df = df.replace("FirstView","1")
    df = df.replace("ThirdView","3")
    
    df['sin_time'] = np.sin(2*np.pi*df.hour/24) #encode circularity of 24 h clock
    df['cos_time'] = np.cos(2*np.pi*df.hour/24)
    df.drop('hour', axis=1, inplace=True)
    
    df['slotvisibility'] = pd.to_numeric(df['slotvisibility'], errors='coerce')
    df['slotvisibility'] = df['slotvisibility'].fillna(avgSlot)
    
    
    
    
    
            
    df = df.drop(['userid'],axis=1)

    
    df['keypage'].fillna("", inplace=True)
    
    df['domain'].fillna("", inplace=True)
    
    df['adexchange'].fillna("", inplace=True)
    


    
    return df
    
    

In [ ]:

train = transformDatas(train, avgSlot)


In [ ]:
for f in train.columns: 
    if train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values)) 
        train[f] = lbl.transform(list(train[f].values))

normalize numeric data

In [ ]:
norms = ['slotwidth', 'slotheight', 'slotvisibility','slotprice', 'slotvisibility', 'sin_time', 'cos_time']

for feature_name in norms:
        max_value = train[feature_name].max()
        min_value = train[feature_name].min()
        train[feature_name] = (train[feature_name] - min_value) / (max_value - min_value)

In [ ]:
#categorical data
cats = ['weekday','useragent','city','adexchange' ,'domain' ,'slotid' , 'slotformat','creative','keypage','advertiser']

In [ ]:

    
train = pd.get_dummies(train,columns=cats, sparse =True)

In [ ]:
tests = train.tail(len(tests))

In [ ]:
train = train.head(len(train)-len(tests))

In [ ]:
valid = train.tail(len(valid))
train = train.head(len(train)-len(valid))

In [ ]:
train = train.to_sparse(fill_value=None)

tests = tests.to_sparse(fill_value=None)

valid = valid.to_sparse(fill_value=None)

In [ ]:
train = train.to_coo()

tests = tests.to_coo()

valid = valid.to_coo()

In [ ]:
from sklearn.model_selection import train_test_split

Y= clicks
x, x_test, y, y_test = train_test_split(train, Y, test_size=0.05, random_state=42, stratify=Y)



In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping

from keras.optimizers import Adam

model = Sequential([
    Dense(32, input_shape=(49261,)),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid'),
])

ads = Adam(lr=0.0005)
model.compile(loss='binary_crossentropy', optimizer=ads, metrics=['accuracy'])  #with default lr-valids got 147 clicks



In [ ]:
len(nBig)/len(small)

call = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

In [ ]:
class_weight = {0: 1.,
                1: 220.}

In [ ]:
x.todense
x_test.todense


In [ ]:
history = model.fit(x, y, batch_size=128, epochs=20, validation_data=(x_test,y_test), class_weight=class_weight, callbacks = [call])  



In [ ]:
valid.todense
valid =valid.tocsr()

In [ ]:
v_pred=model.predict(valid)
v_clicks = cVal.values

roc_auc_score(v_clicks, v_pred)



In [ ]:
avgCtr = 1793/(2429188+1793)

In [ ]:


true = pd.Series(clicks).values
price = pd.Series(price).values

In [ ]:
vTrue = pd.Series(cVal).values
vPrice = pd.Series(vPrice).values

In [ ]:
roc_auc_score(vTrue, v_pred)

In [ ]:
clist = []
highest_pay_price = max(price)+1

def clicksAtPrice(price, bid, y_pred, true):
  clicks = 0
  budget = 6250
  for i, p in enumerate(price):

    if budget >= p/1000 and bid*(y_pred[i]/avgCtr) >= p:
      budget -= p/1000
      clicks += true[i]

  return clicks

mClick = 0
bid = 0
for i in tqdm(range(0, 60)):
    
    a = clicksAtPrice(vPrice,i,v_pred,cVal)
    
    clist.append(a+6)
    
    if a>mClick:
        mClick=a
        bid = i
      
  

In [ ]:
tests =tests.tocsr()

In [ ]:

bid_pred = model.predict(tests)
biid = pd.Series(ids).values

linear bid strategy for test data

In [ ]:
import csv

with open('mlp.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['bidid']  + ['bidprice']) 
    for i in range(0,len(biid)):
        writer.writerow([biid[i]]+ [bid*(bid_pred[i]/avgCtr)])